In [2]:
import pandas as pd
import numpy as np

## All the data frames

In [3]:
A = pd.read_csv('csv/origin_files/FAOSTAT_data_4-20-2021.csv')
B = pd.read_csv('csv/origin_files/FAOSTAT_data_4-20-2021-2.csv')
C = pd.read_csv('csv/origin_files/FAOSTAT_data_4-20-2021-3.csv')
D = pd.read_csv('csv/origin_files/FAOSTAT_data_4-20-2021-4.csv')
E = pd.read_csv('csv/origin_files/FAOSTAT_data_4-20-2021-5.csv')
F = pd.read_excel('tools/tables_relation.xlsx')
G = pd.read_csv('csv/origin_files/environment_impact_of_food_production.csv')
H = pd.read_excel('tools/drop_python.xlsx')

In [4]:
G.head()

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Total_emissions,Eutrophying emissions per 1000kcal (gPO₄eq per 1000kcal),...,Freshwater withdrawals per 100g protein (liters per 100g protein),Freshwater withdrawals per kilogram (liters per kilogram),Greenhouse gas emissions per 1000kcal (kgCO₂eq per 1000kcal),Greenhouse gas emissions per 100g protein (kgCO₂eq per 100g protein),Land use per 1000kcal (m² per 1000kcal),Land use per kilogram (m² per kilogram),Land use per 100g protein (m² per 100g protein),Scarcity-weighted water use per kilogram (liters per kilogram),Scarcity-weighted water use per 100g protein (liters per 100g protein),Scarcity-weighted water use per 1000kcal (liters per 1000 kilocalories)
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1,1.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0,1.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3,1.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,1.6,4.281357,...,371.076923,482.4,0.945482,1.907692,2.897446,7.6,5.846154,18786.2,14450.92308,7162.104461
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,4.0,9.514379,...,3166.760563,2248.4,1.207271,6.267606,0.759631,2.8,3.943662,49576.3,69825.77465,13449.891480


In [5]:
G = G.drop(G.columns[[9,10,11,12,13,14,15,16,17,18,19,20,21,22]],axis=1)

In [6]:
list = [A,B,C,D,E]

In [7]:
for i in list:
    i.drop(['Domain','Domain Code','Element Code','Year Code','Year','Flag Description','Flag'],axis=1,inplace=True)

In [8]:
for i in list:
    display(i.head(2))

,Area Code,Area,Element,Item Code,Item,Unit,Value
0,2,Afghanistan,Import Quantity,231,Almonds shelled,tonnes,3520.0
1,2,Afghanistan,Import Quantity,221,"Almonds, with shell",tonnes,1585.0


,Area Code,Area,Element,Item Code,Item,Unit,Value
0,2,Afghanistan,Export Quantity,862,Alfalfa meal and pellets,tonnes,49.0
1,2,Afghanistan,Export Quantity,231,Almonds shelled,tonnes,6848.0


,Area Code,Area,Element,Item Code,Item,Unit,Value
0,2,Afghanistan,Production,221,"Almonds, with shell",tonnes,38205.0
1,2,Afghanistan,Production,711,"Anise, badian, fennel, coriander",tonnes,19025.0


,Area Code,Area,Element,Item Code,Item,Unit,Value
0,2,Afghanistan,Production,221,"Almonds, with shell",tonnes,38205.0
1,2,Afghanistan,Production,711,"Anise, badian, fennel, coriander",tonnes,19025.0


,Area Code,Area,Element,Item Code,Item,Unit,Value
0,2,Afghanistan,Production,1062,"Eggs, hen, in shell",tonnes,22159.0
1,2,Afghanistan,Production,1067,"Eggs, hen, in shell (number)",1000 No,443181.0


## Joining dataframes

In [9]:
CDE = pd.concat([C,D,E],axis=0)

In [10]:
AB = pd.merge(left = A,
                         right = B,
                         how = 'left', 
                         left_on = ['Item Code','Area','Area Code'], 
                         right_on= ['Item Code','Area','Area Code'])

In [11]:
production = pd.merge(left = AB,
                         right = CDE,
                         how = 'left', 
                         left_on = ['Item Code','Area','Area Code'], 
                         right_on= ['Item Code','Area','Area Code'])

In [12]:
production = production.drop_duplicates()

In [13]:
production = production.drop(['Area Code','Item_y','Unit_x','Unit_y','Unit','Item'], axis=1)

In [14]:
production = production.rename(columns={'Value_x':'import_quantity','Value_y':'export_quantity','Value':'production','Item_x':'food_product'})
production = production.drop(['Element_x','Element_y','Element'],axis=1)

In [15]:
# Doing joins, it is posible that some countries does not produce anything but they export or import something, or
# just the oposite.
production = production.fillna(0)

## Joining production, importation and exportation tables.

In [16]:
join1 = pd.merge(left = production,
                         right = F,
                         how = 'left', 
                         left_on = 'food_product', 
                         right_on= 'production')

In [17]:
join2 = pd.merge(left = join1,
                         right = G,
                         how = 'left', 
                         left_on = 'emissions', 
                         right_on= 'Food product')

In [18]:
data = join2.drop(['production_y','emissions','Food product'],axis=1)

In [19]:
#Removing catefories which are not food.
drop = H['drop']
drop = [i for i in drop]
data = (data[~data['food_product'].isin(drop)])

In [20]:
data['import_emisions'] =(data['import_quantity']*1000)*data['Total_emissions']
data['export_emisions'] =(data['export_quantity']*1000)*data['Total_emissions']

In [21]:
for i in data.columns[-10:-2]:
    data[i] = data[i]*1000*data['production_x']

In [22]:
data['consumption_emisions'] =data['Total_emissions']-data['import_emisions']+data['export_emisions']

In [26]:
#Exporting de csv file.
data.to_csv('csv/food_products_emission.csv', index=False)

In [24]:
#Final resoult
data.pivot_table(index=['Area','food_product'])

Animal Feed        Farm  \
Area        food_product                                                
Afghanistan Almonds shelled                           0.0         0.0   
            Almonds, with shell                       0.0  80230500.0   
            Anise, badian, fennel, coriander          0.0  26635000.0   
            Apples                                    0.0  50064800.0   
            Apricots                                  0.0  51745200.0   
...                                                   ...         ...   
Zimbabwe    Wafers                                    0.0         0.0   
            Walnuts, shelled                          0.0         0.0   
            Wheat                                     0.0  64000000.0   
            Wine                                      0.0         0.0   
            Yoghurt, concentrated or not              0.0         0.0   

                                              Item Code  Land use change  \
Area        food_product                                                   
Afghanistan Almonds shelled                         231              0.0   
            Almonds, with shell                     221      -80230500.0   
            Anise, badian, fennel, coriander        711        7610000.0   
            Apples                                  515              0.0   
            Apricots                                526       12936300.0   
...                                                 ...              ...   
Zimbabwe    Wafers                                  110              0.0   
            Walnuts, shelled                        232              0.0   
            Wheat                                    15        8000000.0   
            Wine                                    564              0.0   
            Yoghurt, concentrated or not            892              0.0   

                                               Packging  Processing  \
Area        food_product                                              
Afghanistan Almonds shelled                         0.0         0.0   
            Almonds, with shell               3820500.0         0.0   
            Anise, badian, fennel, coriander  1902500.0   7610000.0   
            Apples                                  0.0         0.0   
            Apricots                                0.0         0.0   
...                                                 ...         ...   
Zimbabwe    Wafers                                  0.0         0.0   
            Walnuts, shelled                        0.0         0.0   
            Wheat                             8000000.0  16000000.0   
            Wine                                    0.0         0.0   
            Yoghurt, concentrated or not            0.0         0.0   

                                                 Retail  Total_emissions  \
Area        food_product                                                   
Afghanistan Almonds shelled                         0.0              0.0   
            Almonds, with shell                     0.0        7641000.0   
            Anise, badian, fennel, coriander        0.0       45660000.0   
            Apples                                  0.0       75097200.0   
            Apricots                                0.0       90554100.0   
...                                                 ...              ...   
Zimbabwe    Wafers                                  0.0              0.0   
            Walnuts, shelled                        0.0              0.0   
            Wheat                             8000000.0      112000000.0   
            Wine                                    0.0              0.0   
            Yoghurt, concentrated or not            0.0              0.0   

                                               Transport  \
Area        food_product                                   
Afghanistan Almonds shelled                          0.0   
            Al